In [ ]:
# %conda list -n musicnlp
# %pip install lyricwikia
# %conda install requests 
# %pip install lyricsgenius

In [ ]:
import os
import json
import lyricsgenius
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
from tqdm import tqdm
from pathlib import Path

# Genius API setup
genius = lyricsgenius.Genius("aixLdY9NuA2j7Y4_v49guDLJqzQtGZZcZHyt_rW5NN_UbtOkHRM7YnNYxIzOvMryE-j2UFGhhVWjMYtgIDlgEA")

# Spotify API setup
spotify = spotipy.Spotify(client_credentials_manager=SpotifyClientCredentials(client_id="dfe2577ba1c1433b9b4b60402fc3d81d", client_secret="9492d94136224f83b7f0c19a41109b5e"))
client_credentials_manager = SpotifyClientCredentials(client_id="dfe2577ba1c1433b9b4b60402fc3d81d", client_secret="9492d94136224f83b7f0c19a41109b5e")
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)
# Define your root directory
root_dir = 'archive-new'

# Directory for JSON files
json_dir = Path('archive-new/json')
json_dir.mkdir(parents=True, exist_ok=True)

# Function to get Spotify metadata
def get_spotify_metadata(artist, track):
    results = sp.search(q='artist:' + artist + ' track:' + track, type='track')
    items = results['tracks']['items']
    if len(items) > 0:
        track_id = items[0]['id']
        track_features = sp.audio_features(track_id)[0]
        return {
            'energy': track_features['energy'],
            'acousticness': track_features['acousticness'],
            'danceability': track_features['danceability'],
            'instrumentalness': track_features['instrumentalness'],
            'liveness': track_features['liveness'],
            'speechiness': track_features['speechiness'],
            'loudness': track_features['loudness'],
            'tempo': track_features['tempo'],
            'time_signature': track_features['time_signature'],
            'valence': track_features['valence'],
            'genre': sp.artist(items[0]['artists'][0]['id'])['genres'],
            'popularity': items[0]['popularity']
        }
    return {}

# Walk through the directory structure
for subdir, dirs, files in tqdm(os.walk(root_dir), desc='Walking through directories'):
    for file in tqdm(files, desc=f'Processing files in {subdir}'):
        if file.endswith('.mid'):
            # Parse out the artist/band name and song title
            artist_name = os.path.basename(subdir).replace('_', ' ')
            song_title = os.path.splitext(file)[0].replace('_', ' ')
            
            # Retrieve lyrics
            # Your implementation here
            try:
                song = genius.search_song(song_title, artist_name)
                lyrics = song.lyrics if song else "Lyrics not found"
            except Exception as e:
                lyrics = f"Error retrieving lyrics: {e}"
            
            # Retrieve Spotify metadata
            # Your implementation here
            spotify_metadata = get_spotify_metadata(artist_name, song_title)
            
            # Create song data dictionary
            song_data = {
                'midi_file_path': os.path.join(subdir, file),
                'artist_band_name': artist_name,
                'song_title': song_title,
                'lyrics': lyrics,
                'spotify_metadata': spotify_metadata
            }

            # Construct a filename for the JSON file
            json_filename = f"{artist_name}_{song_title}.json".replace(" ", "_")
            json_filepath = json_dir / json_filename

            # Save the song data to a JSON file
            with open(json_filepath, 'w') as outfile:
                json.dump(song_data, outfile, indent=4)

print("Data collection completed.")


In [ ]:
import os
import json
from pathlib import Path

def is_valid_data(data):
    # Check if the lyrics and Spotify metadata are valid
    if 'lyrics' not in data or 'spotify_metadata' not in data:
        return False
    if data['lyrics'].startswith("Error") or not data['lyrics']:
        return False
    if not data['spotify_metadata']:
        return False
    return True

def segregate_lyrics(lyrics):
    # Segregate lyrics into verses, chorus/prechorus, and bridge
    # Modify this function as per the specific format of your lyrics
    sections = {'Verse': [], 'Chorus': [], 'Pre-Chorus': [], 'Bridge': []}
    current_section = None
    for line in lyrics.split('\n'):
        if '[Verse' in line:
            current_section = 'Verse'
        elif '[Chorus' in line or '[Pre-Chorus' in line:
            current_section = 'Chorus'
        elif '[Bridge' in line:
            current_section = 'Bridge'
        elif current_section:
            sections[current_section].append(line)
    return sections

# Path to the folder containing JSON files
json_folder_path = Path('archive-new/json')

# Iterate over each JSON file in the folder
for json_file in json_folder_path.glob('*.json'):
    with open(json_file, 'r', encoding='utf-8') as file:
        data = json.load(file)

    # Check if the data is valid
    if not is_valid_data(data):
        # Delete the file if data is invalid
        os.remove(json_file)
        continue

    # If lyrics exist, segregate them
    if 'lyrics' in data and data['lyrics']:
        data['lyrics'] = segregate_lyrics(data['lyrics'])

    # Save the updated data back to the file
    with open(json_file, 'w', encoding='utf-8') as file:
        json.dump(data, file, indent=4)

print("Processing complete.")
